測試一張OCR

In [ ]:
from paddleocr import PaddleOCR, draw_ocr
from PIL import Image

ocr = PaddleOCR(use_angle_cls=True, lang='ch', use_gpu=False)

image_path = r'C:\Users\zhuang\Desktop\Paddle\img\id_card_images\image_00000.png'
result = ocr.ocr(image_path, cls=True)

if result and result[0]:
    image = Image.open(image_path).convert('RGB')
    boxes = [line[0] for line in result[0]]
    txts = [line[1][0] for line in result[0]]
    scores = [line[1][1] for line in result[0]]

    try:
        font_path_for_drawing = r'C:\Windows\Fonts\msyh.ttc'  
        im_show = draw_ocr(image, boxes, txts, scores, font_path=font_path_for_drawing)
        im_show = Image.fromarray(im_show)
        output_image_path = 'result.jpg'
        im_show.save(output_image_path)
        print(f"\n識別結果已保存到可視化圖片：{output_image_path}")

    except ImportError:
        print("\n警告：無法繪製識別結果。請安裝 'opencv-python' 和 'Pillow' 庫 (pip install opencv-python Pillow)。")
    except FileNotFoundError:
        print(f"\n錯誤：繪製圖片時找不到字體文件。請檢查 font_path: '{font_path_for_drawing}' 是否正確。")
    except Exception as e:
        print(f"\n繪製結果時發生未知錯誤: {e}")
else:
    print("未在圖片中識別到任何文字。")

print("\nOCR 測試完成。")


ImportError: cannot import name 'draw_ocr' from 'paddleocr' (c:\Users\zhuang\Desktop\Paddle\paddleocr\__init__.py)

OCR整個資料夾 img\id_card_images


In [ ]:
from paddleocr_backup import PaddleOCR, draw_ocr
import os
from PIL import Image 


input_image_dir = r'C:\Users\User\Desktop\PaddleOCR\img\id_card_images'
output_ocr_image_dir = r'C:\Users\User\Desktop\PaddleOCR\img\ocrimg1'
output_ocr_text_dir = r'C:\Users\User\Desktop\PaddleOCR\img\ocrtxt1'


if not os.path.exists(input_image_dir):
    print(f"錯誤：輸入圖片資料夾 '{input_image_dir}' 不存在。請檢查路徑是否正確。")
    exit() 


os.makedirs(output_ocr_image_dir, exist_ok=True)
os.makedirs(output_ocr_text_dir, exist_ok=True)

print(f"將從 '{input_image_dir}' 讀取圖片。")
print(f"OCR 可視化圖片將保存到： {output_ocr_image_dir}")
print(f"OCR 識別文字將保存到： {output_ocr_text_dir}")

# lang='ch' 表示使用中文模型，'en' 表示英文
ocr = PaddleOCR(use_angle_cls=True, lang="ch", use_gpu=False)
font_path_for_drawing = r'C:\Windows\Fonts\msyh.ttc'
image_files = [f for f in os.listdir(input_image_dir)
               if os.path.isfile(os.path.join(input_image_dir, f)) and
               f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]

if not image_files:
    print(f"在 '{input_image_dir}' 中沒有找到任何圖片文件。")
else:
    print(f"\n在 '{input_image_dir}' 中找到 {len(image_files)} 張圖片，開始進行 OCR 處理...")

    
    processed_count = 0

    for img_filename in sorted(image_files): 
        full_image_path = os.path.join(input_image_dir, img_filename)

        print(f"\n正在處理圖片：'{img_filename}' (總計 {processed_count + 1}/{len(image_files)})")

        try:
            
            result = ocr.ocr(full_image_path, cls=True)

            recognized_texts = []
            if result and result[0]:
                for line in result[0]:
                    text = line[1][0] 
                    recognized_texts.append(text)

                # 保存結果
                output_text_filename = f"{processed_count}.txt" 
                output_text_path = os.path.join(output_ocr_text_dir, output_text_filename)

                with open(output_text_path, 'w', encoding='utf-8') as f:
                    for text_line in recognized_texts:
                        f.write(text_line + '\n')
                print(f"  識別文字已保存到：{output_text_path}")

            
                try:
                    img = Image.open(full_image_path).convert('RGB')
                    boxes = [line[0] for line in result[0]]
                    txts = [line[1][0] for line in result[0]]
                    scores = [line[1][1] for line in result[0]]

                    im_show = draw_ocr(img, boxes, txts, scores, font_path=font_path_for_drawing)
                    im_show = Image.fromarray(im_show)

                    output_image_filename = f"{processed_count}.jpg" # 命名為 0.jpg, 1.jpg...
                    output_image_path = os.path.join(output_ocr_image_dir, output_image_filename)
                    im_show.save(output_image_path)
                    print(f"  可視化圖片已保存到：{output_image_path}")

                except ImportError:
                    print("\n警告：無法繪製識別結果。請安裝 'opencv-python' 和 'Pillow' 庫 (pip install opencv-python Pillow)。")
                except FileNotFoundError:
                    print(f"\n錯誤：繪製圖片時找不到字體文件。請檢查 font_path: '{font_path_for_drawing}' 是否正確。")
                except Exception as e:
                    print(f"\n繪製結果時發生未知錯誤: {e}")
            else:
                print(f"  未在圖片 '{img_filename}' 中識別到任何文字。")

            processed_count += 1 

        except Exception as e:
            print(f"處理圖片 '{img_filename}' 時發生錯誤: {e}")

    print("\n所有圖片的 OCR 處理完成！")

ModuleNotFoundError: No module named 'paddleocr_backup'

正則化ocrimg1資料夾並將正規化後存到ocrtxt_normalized

In [ ]:
import os
import re
from pathlib import Path

def clean_ocr_text(text):
    """
    清理OCR文本，移除字段标签，只保留数据
    """
    # 定义需要移除的字段标签
    field_labels = [
        '姓名', '性别', '民族', '出生', '住址', '公民身份号码',
        '年', '月', '日'
    ]
    
    # 按行分割文本
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    cleaned_lines = []
    
    for line in lines:
        cleaned_line = line
        
        # 移除字段标签
        for label in field_labels:
            cleaned_line = cleaned_line.replace(label, '')
        
        # 处理民族简化（汉族->汉，回族->回等）
        cleaned_line = re.sub(r'汉族', '汉', cleaned_line)
        cleaned_line = re.sub(r'回族', '回', cleaned_line)
        cleaned_line = re.sub(r'满族', '满', cleaned_line)
        cleaned_line = re.sub(r'蒙古族', '蒙古', cleaned_line)
        
        # 处理日期格式，将"1988年12月21日"分离为独立的年、月、日
        date_pattern = r'(\d{4})年(\d{1,2})月(\d{1,2})日'
        date_match = re.search(date_pattern, cleaned_line)
        if date_match:
            year, month, day = date_match.groups()
            cleaned_lines.extend([year, month, day])
            continue
            
        # 清理多余的空格和特殊字符
        cleaned_line = re.sub(r'\s+', '', cleaned_line)  # 移除所有空格
        cleaned_line = cleaned_line.strip()
        
        # 如果清理后还有内容，就添加到结果中
        if cleaned_line:
            cleaned_lines.append(cleaned_line)
    
    return '\n'.join(cleaned_lines)

def process_ocr_files(input_dir, output_dir):
    """
    批量处理OCR文本文件
    """
    # 创建输出目录（如果不存在）
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # 统计信息
    processed_count = 0
    error_count = 0
    
    print(f"开始处理文件...")
    print(f"输入目录: {input_dir}")
    print(f"输出目录: {output_dir}")
    print("-" * 50)
    
    # 遍历输入目录中的所有txt文件
    for filename in os.listdir(input_dir):
        if filename.lower().endswith('.txt'):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename)
            
            try:
                # 读取原始文件
                with open(input_path, 'r', encoding='utf-8') as f:
                    original_text = f.read()
                
                # 清理文本
                cleaned_text = clean_ocr_text(original_text)
                
                # 写入清理后的文件
                with open(output_path, 'w', encoding='utf-8') as f:
                    f.write(cleaned_text)
                
                print(f"✅ 处理完成: {filename}")
                print(f"   原始行数: {len(original_text.split())}")
                print(f"   清理后行数: {len(cleaned_text.split())}")
                print()
                
                processed_count += 1
                
            except Exception as e:
                print(f"❌ 处理错误: {filename} - {str(e)}")
                error_count += 1
    
    print("-" * 50)
    print(f"处理完成！")
    print(f"成功处理: {processed_count} 个文件")
    print(f"处理失败: {error_count} 个文件")

def preview_cleaning(input_dir, max_files=3):
    """
    预览清理效果，显示前几个文件的处理前后对比
    """
    print("📋 清理效果预览")
    print("=" * 60)
    
    file_count = 0
    for filename in os.listdir(input_dir):
        if filename.lower().endswith('.txt') and file_count < max_files:
            input_path = os.path.join(input_dir, filename)
            
            try:
                with open(input_path, 'r', encoding='utf-8') as f:
                    original_text = f.read()
                
                cleaned_text = clean_ocr_text(original_text)
                
                print(f"\n📄 文件: {filename}")
                print("-" * 30)
                print("处理前:")
                print(original_text[:200] + ("..." if len(original_text) > 200 else ""))
                print("-" * 30)
                print("处理后:")
                print(cleaned_text)
                print("=" * 60)
                
                file_count += 1
                
            except Exception as e:
                print(f"❌ 预览错误: {filename} - {str(e)}")

if __name__ == "__main__":
    # --- 設定輸入和輸出資料夾路徑 ---
    input_ocrtxt_dir = r'C:\Users\zhuang\Desktop\Paddle\img\ocrtxt1'
    output_normalized_ocrtxt_dir = r'C:\Users\zhuang\Desktop\Paddle\img\ocrtxt_normalized'
    
    # 检查输入目录是否存在
    if not os.path.exists(input_ocrtxt_dir):
        print(f"❌ 错误: 输入目录不存在: {input_ocrtxt_dir}")
        exit(1)
    

    # 开始批量处理
    process_ocr_files(input_ocrtxt_dir, output_normalized_ocrtxt_dir)
    
    print(f"\n所有处理完成的文件保存在: {output_normalized_ocrtxt_dir}")

开始处理文件...
输入目录: C:\Users\zhuang\Desktop\Paddle\img\ocrtxt
输出目录: C:\Users\zhuang\Desktop\Paddle\img\ocrtxt_normalized
--------------------------------------------------
✅ 处理完成: 0.txt
   原始行数: 9
   清理后行数: 6

✅ 处理完成: 1.txt
   原始行数: 13
   清理后行数: 8

✅ 处理完成: 10.txt
   原始行数: 12
   清理后行数: 7

✅ 处理完成: 100.txt
   原始行数: 10
   清理后行数: 7

✅ 处理完成: 101.txt
   原始行数: 10
   清理后行数: 6

✅ 处理完成: 102.txt
   原始行数: 11
   清理后行数: 7

✅ 处理完成: 103.txt
   原始行数: 10
   清理后行数: 6

✅ 处理完成: 104.txt
   原始行数: 9
   清理后行数: 6

✅ 处理完成: 105.txt
   原始行数: 11
   清理后行数: 6

✅ 处理完成: 106.txt
   原始行数: 11
   清理后行数: 6

✅ 处理完成: 107.txt
   原始行数: 10
   清理后行数: 5

✅ 处理完成: 108.txt
   原始行数: 12
   清理后行数: 7

✅ 处理完成: 109.txt
   原始行数: 12
   清理后行数: 7

✅ 处理完成: 11.txt
   原始行数: 10
   清理后行数: 5

✅ 处理完成: 110.txt
   原始行数: 13
   清理后行数: 6

✅ 处理完成: 111.txt
   原始行数: 9
   清理后行数: 6

✅ 处理完成: 112.txt
   原始行数: 12
   清理后行数: 7

✅ 处理完成: 113.txt
   原始行数: 9
   清理后行数: 6

✅ 处理完成: 114.txt
   原始行数: 11
   清理后行数: 6

✅ 处理完成: 115.txt
   原始行数: 10
   清理后行数: 6

✅ 处理完成: 116.txt
   原始行

CER

In [1]:
import os
import re
import Levenshtein

# 資料夾路徑
gt_dir = r"C:\Users\zhuang\Desktop\Paddle\img\id_card_ocr_texts"
pred_dir = r"C:\Users\zhuang\Desktop\Paddle\img\ocrtxt_normalized"

# 用來過濾空白與換行字元的函數
def clean_text(text):
    return re.sub(r'\s+', '', text)  # 移除所有空白、換行、Tab 等

# 統計變數
total_distance = 0
total_chars = 0
missing_pred_files = []

print("每個檔案的 CER 結果：\n")

for gt_filename in os.listdir(gt_dir):
    if not gt_filename.endswith(".txt"):
        continue

    # ground truth 檔名如：ocr_00000.txt -> 對應預測檔名：image_00000.txt
    index = gt_filename.replace("ocr_", "").replace(".txt", "")
    pred_filename = f"image_{index}.txt"

    gt_path = os.path.join(gt_dir, gt_filename)
    pred_path = os.path.join(pred_dir, pred_filename)

    if not os.path.exists(pred_path):
        missing_pred_files.append(pred_filename)
        continue

    with open(gt_path, "r", encoding="utf-8") as f_gt, open(pred_path, "r", encoding="utf-8") as f_pred:
        gt_text = clean_text(f_gt.read())
        pred_text = clean_text(f_pred.read())

        distance = Levenshtein.distance(pred_text, gt_text)
        char_count = len(gt_text)
        file_cer = distance / char_count if char_count > 0 else 0

        print(f"{gt_filename} 對應 {pred_filename}：CER = {file_cer:.4f}")

        total_distance += distance
        total_chars += char_count

# 輸出總 CER
if total_chars > 0:
    total_cer = total_distance / total_chars
    print(f"\n🔢 總 CER：{total_cer:.4f}")
else:
    print("❗ 無有效的 ground truth 字元可計算 CER。")

# 列出缺少預測檔案
if missing_pred_files:
    print("\n⚠️ 找不到以下預測檔案：")
    for f in missing_pred_files:
        print(f" - {f}")


每個檔案的 CER 結果：

ocr_00000.txt 對應 image_00000.txt：CER = 0.0250
ocr_00002.txt 對應 image_00002.txt：CER = 0.1628
ocr_00006.txt 對應 image_00006.txt：CER = 0.0556
ocr_00007.txt 對應 image_00007.txt：CER = 0.0256
ocr_00008.txt 對應 image_00008.txt：CER = 0.0000
ocr_00012.txt 對應 image_00012.txt：CER = 0.0000
ocr_00016.txt 對應 image_00016.txt：CER = 0.0256
ocr_00018.txt 對應 image_00018.txt：CER = 0.0000
ocr_00020.txt 對應 image_00020.txt：CER = 0.0000
ocr_00021.txt 對應 image_00021.txt：CER = 0.1316
ocr_00022.txt 對應 image_00022.txt：CER = 0.1282
ocr_00023.txt 對應 image_00023.txt：CER = 0.0000
ocr_00024.txt 對應 image_00024.txt：CER = 0.0000
ocr_00025.txt 對應 image_00025.txt：CER = 0.1026
ocr_00026.txt 對應 image_00026.txt：CER = 0.0238
ocr_00027.txt 對應 image_00027.txt：CER = 0.0000
ocr_00028.txt 對應 image_00028.txt：CER = 0.0000
ocr_00030.txt 對應 image_00030.txt：CER = 0.0000
ocr_00035.txt 對應 image_00035.txt：CER = 0.0000
ocr_00036.txt 對應 image_00036.txt：CER = 0.0250
ocr_00037.txt 對應 image_00037.txt：CER = 0.0222
ocr_00040.txt 對應 im